## JSSP Tabu Search and Genetic Algorithm Example

This notebook demonstrates how to use both parallel tabu seach and the genetic algorithm.

In [1]:
from JSSP import CSVData

# initialize data
data_directory = '../data/given_data'
instance_data = CSVData(data_directory + '/sequenceDependencyMatrix.csv',
                              data_directory + '/machineRunSpeed.csv',
                              data_directory + '/jobTasks.csv')

# uncomment this to print the data that was read in
# print(instance_data)

In [2]:
from JSSP.genetic_algorithm import GASelectionEnum

output_dir = './example_output'
output_results = True
auto_open = True
verbose = True
benchmark = True
progress_bar = True

# TS parameters
ts_runtime = 30 # in seconds
num_processes = 4
num_solutions_per_process = 5
tabu_list_size = 50
neighborhood_size = 200
neighborhood_wait = 0.12
probability_change_machine = 0.8
reset_threshold = 100

# ga parameters
ga_runtime = 30 # in seconds
mutation_probability = 0.2
population_size = 100
selection_method = GASelectionEnum.FITNESS_PROPORTIONATE
selection_size = 5
population = []

### Tabu Search Before Genetic Algorithm

In [3]:
from JSSP.solver import Solver

# create solver
solver = Solver(instance_data)

# run tabu seach
solver.tabu_search_time(runtime=ts_runtime,
                        num_processes=num_processes,
                        num_solutions_per_process=num_solutions_per_process,
                        tabu_list_size=tabu_list_size,
                        neighborhood_size=neighborhood_size,
                        neighborhood_wait=neighborhood_wait,
                        probability_change_machine=probability_change_machine,
                        reset_threshold=reset_threshold,
                        benchmark=benchmark,
                        verbose=verbose,
                        progress_bar=progress_bar
                        )

# add all tabu search solutions to population
for ts_agent in solver.ts_agent_list:
    population += ts_agent.all_solutions
    
# run genetic algorithm
solution = solver.genetic_algorithm_time(runtime=ga_runtime,
                                         population=population,
                                         population_size=population_size,
                                         selection_method_enum=selection_method,
                                         selection_size=selection_size,
                                         mutation_probability=mutation_probability, 
                                         benchmark=benchmark,
                                         verbose=verbose,
                                         progress_bar=progress_bar
                                        )

print('\nSolution:')
print(solution)

if output_results:
    if benchmark:
        solver.output_benchmark_results(output_dir, name='example_benchmark', auto_open=auto_open)
    else:
        solution.create_schedule_xlsx_file(output_dir)

Running benchmark of TS
Parameters:
stopping_condition = 30
time_condition = True
num_solutions_per_process = 5
num_processes = 4
tabu_list_size = 50
neighborhood_size = 200
neighborhood_wait = 0.12
probability_change_machine = 0.8
reset_threshold = 100

Initial Solution's makespans:
[8472, 9358, 8268, 7362]

child TS process started. pid = 1535
child TS process started. pid = 1536
child TS process started. pid = 1538
child TS process started. pid = 1539


||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:00:29


child TS process finished. pid = 1535
child TS process finished. pid = 1536
child TS process finished. pid = 1538
child TS process finished. pid = 1539
Running benchmark of GA
Parameters:
stopping_condition = 30
time_condition = True
population_size = 100
selection_method = _fitness_proportionate_selection
mutation_probability = 0.2


||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:00:29



Solution
makespan = 4057.698795180723
machine_makespans = [4052.7235772357726, 4002.913793103449, 4056.684731406831, 4057.698795180723, 4050.585585585586, 4050.48, 4041.7006392918615, 4057.6092715231794]
operation_list =
[[ 0  0  0  4]
 [14  1  0  6]
 [47  0  0  3]
 [46  0  0  4]
 [37  0  0  2]
 [46  1  0  1]
 [35  0  0  2]
 [29  1  0  0]
 [36  0  0  5]
 [ 3  0  0  0]
 [24  1  0  6]
 [15  0  0  5]
 [48  0  0  0]
 [ 4  2  0  0]
 [43  0  0  7]
 [40  0  0  3]
 [28  1  0  5]
 [45  1  0  1]
 [45  2  0  3]
 [12  0  0  4]
 [39  0  0  6]
 [ 1  0  0  3]
 [ 9  1  0  4]
 [31  0  0  0]
 [41  0  0  6]
 [15  1  0  5]
 [17  0  0  7]
 [26  0  0  4]
 [29  0  0  1]
 [32  0  0  6]
 [37  3  1  4]
 [ 4  1  0  4]
 [ 6  2  0  7]
 [ 2  0  0  4]
 [11  0  0  5]
 [ 3  1  1  0]
 [37  2  1  7]
 [20  2  0  7]
 [30  0  0  7]
 [10  0  0  2]
 [ 7  0  0  7]
 [22  0  0  7]
 [14  0  0  0]
 [ 6  0  0  1]
 [37  1  1  7]
 [34  2  0  1]
 [30  3  1  6]
 [29  2  1  6]
 [ 5  0  0  5]
 [35  1  1  7]
 [42  0  0  3]
 [48  1  1  7

### Genetic Algorithm Before Tabu Search

In [4]:
from JSSP.solver import Solver

# create solver
solver = Solver(instance_data)

# run genetic algorithm
solver.genetic_algorithm_time(runtime=ga_runtime, 
                              population_size=population_size, 
                              selection_method_enum=selection_method,
                              selection_size=selection_size,
                              mutation_probability=mutation_probability, 
                              benchmark=benchmark,
                              verbose=verbose,
                              progress_bar=progress_bar
                              )

# pass best solutions found to tabu search
initial_solutions = sorted(solver.ga_agent.result_population)[:num_processes]

# run tabu seach
solution = solver.tabu_search_time(runtime=ts_runtime,
                                   num_processes=num_processes,
                                   num_solutions_per_process=num_solutions_per_process,
                                   tabu_list_size=tabu_list_size,
                                   neighborhood_size=neighborhood_size,
                                   neighborhood_wait=neighborhood_wait,
                                   probability_change_machine=probability_change_machine,
                                   reset_threshold=reset_threshold,
                                   initial_solutions=initial_solutions,
                                   benchmark=benchmark,
                                   verbose=verbose,
                                   progress_bar=progress_bar
                                  )
    
print('\nSolution:')
print(solution)

if output_results:
    if benchmark:
        solver.output_benchmark_results(output_dir, name='example_benchmark', auto_open=auto_open)
    else:
        solution.create_schedule_xlsx_file(output_dir)

Running benchmark of GA
Parameters:
stopping_condition = 30
time_condition = True
population_size = 100
selection_method = _fitness_proportionate_selection
mutation_probability = 0.2


||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:00:29


Running benchmark of TS
Parameters:
stopping_condition = 30
time_condition = True
num_solutions_per_process = 5
num_processes = 4
tabu_list_size = 50
neighborhood_size = 200
neighborhood_wait = 0.12
probability_change_machine = 0.8
reset_threshold = 100

Initial Solution's makespans:
[4225, 4225, 4225, 4225]

child TS process started. pid = 2017
child TS process started. pid = 2018
child TS process started. pid = 2025
child TS process started. pid = 2026


||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:00:29


child TS process finished. pid = 2017
child TS process finished. pid = 2018
child TS process finished. pid = 2025
child TS process finished. pid = 2026

Solution:
makespan = 4059.027027027026
machine_makespans = [4051.8455284552856, 4023.4999999999995, 3991.2500000000005, 4054.1686746987953, 4059.027027027026, 4026.11, 4050.132653061224, 4038.7019867549675]
operation_list =
[[30  0  0  7]
 [29  0  0  2]
 [38  0  0  7]
 [39  1  0  7]
 [ 5  0  0  4]
 [27  0  0  4]
 [17  0  0  7]
 [26  0  0  4]
 [ 6  0  0  6]
 [11  0  0  5]
 [ 2  0  0  0]
 [ 8  1  0  2]
 [43  0  0  7]
 [33  0  0  6]
 [ 2  3  1  2]
 [14  0  0  6]
 [37  0  0  2]
 [30  4  1  2]
 [ 1  0  0  3]
 [14  1  0  1]
 [30  2  1  1]
 [ 5  1  1  1]
 [ 0  0  0  4]
 [23  0  0  3]
 [11  2  1  7]
 [31  0  0  0]
 [46  0  0  4]
 [14  2  1  6]
 [45  2  0  7]
 [41  0  0  5]
 [49  0  0  7]
 [42  0  0  3]
 [15  0  0  2]
 [12  0  0  4]
 [43  1  1  1]
 [16  0  0  4]
 [17  1  1  7]
 [26  1  1  6]
 [44  0  0  0]
 [10  0  0  6]
 [25  1  0  7]
 [36  1 